###Preproccessing

In [12]:
lyrics_column = df.iloc[:, 2]

import re
def clean_lyrics(text):
    text = text.encode("ascii", "ignore").decode()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9.,!?\'"\n ]', '', text)
    return text.strip()

cleaned_lyrics = lyrics_column.apply(clean_lyrics)
print(cleaned_lyrics.head())

0    It ain't easy to make money Whoo So now everyb...
1    Uh, yeah, 50 Cent It's Murda Mix Tape Vol. 3 W...
2    Yo yo yo... what up everybody this is fiddy' c...
3    Gunshots 50 Cent... Shady... Aftermath... the ...
4    Yea, we over here, with some fat chicks, haha ...
Name: I change places, to prevent catchin' the cases\nRaces, in the faces, hall at you laces\nThis is a hit, let's see if homicide trace this\n\nThe only thing hotter than my flow is the block (inhale and exhale)\nThat's why I left this snow biz, and got into show biz\nLet's get this clear, it ain't on till I say it's on, (pause), it's on\nI'm eatin', y'all niggas fastin' like it's Rimadon\nBowlish way in Lebanon, know 50 the bomb\nI be at the edge of the bar, sippin' a Don\nI keep the bottle just in case, you never know when it's on\nThis worries bump, I can't go wrong, my team's too strong\nYou want war? I take you to war, now that my money long\nWhy you broke? cat's buy the by lines and fantasize\nThe way I'm spi

In [13]:
with open("cleaned_lyrics.txt", "w", encoding="utf-8") as f:
    f.write("\n\n".join(cleaned_lyrics))

print("✅ Cleaned lyrics saved to cleaned_lyrics.txt")

import os
os.environ["WANDB_DISABLED"] = "true"

✅ Cleaned lyrics saved to cleaned_lyrics.txt


###Fine Tuning

In [14]:
!pip install transformers datasets

In [12]:
from transformers import GPT2Tokenizer
from datasets import Dataset

# Step 1: Load and set up tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 has no pad token by default

# Step 2: Read the lyrics line-by-line
with open("cleaned_lyrics.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

# Step 3: Tokenize each line individually and convert to list of dicts
tokenized_lines = tokenizer(lines, padding="max_length", truncation=True, max_length=512)

# Step 4: Add 'labels' = 'input_ids'
data = []
for i in range(len(tokenized_lines["input_ids"])):
    data.append({
        "input_ids": tokenized_lines["input_ids"][i],
        "attention_mask": tokenized_lines["attention_mask"][i],
        "labels": tokenized_lines["input_ids"][i]  # same as input_ids
    })

# Step 5: Convert into Hugging Face dataset
dataset = Dataset.from_list(data)


In [13]:
#prepare cleaned lyrics into a format that the Hugging Face Trainer can use
from datasets import Dataset
dataset = Dataset.from_dict(tokens)
print(dataset)

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1
})


In [14]:
# Set Training Arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    save_steps=1000,
    logging_dir="./logs",
    logging_steps=100,
    fp16=True,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [15]:
# Train the Model
from transformers import GPT2LMHeadModel
from transformers import Trainer

model = GPT2LMHeadModel.from_pretrained("gpt2")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

import os
os.environ["WANDB_DISABLED"] = "true"
trainer.train()


ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")

print("✅ Fine-tuned model and tokenizer saved.")

In [ ]:
# Generate song lyrics
model = GPT2LMHeadModel.from_pretrained("./fine_tuned_gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("./fine_tuned_gpt2")

input_text = "I'm walking through the city streets,"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output = model.generate(input_ids, max_length=200, num_return_sequences=1, no_repeat_ngram_size=2, temperature=0.7)

generated_lyrics = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_lyrics)

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset

# Step 1: Tokenizer setup
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # important for GPT-2

# Step 2: Read and prepare data
with open("cleaned_lyrics.txt", "r", encoding="utf-8") as file:
    lines = [line.strip() for line in file if line.strip()]  # remove blank lines

# Step 3: Tokenize the data
tokenized = tokenizer(lines, padding="max_length", truncation=True, max_length=512)

# Step 4: Add labels (required by Trainer)
tokenized["labels"] = tokenized["input_ids"]

# Step 5: Convert to Dataset
dataset = Dataset.from_dict(tokenized)

# Step 6: Model setup
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Step 7: Training setup
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    num_train_epochs=1,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="no",  # skip saving checkpoints if not needed
)

# Step 8: Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

# Step 9: Train!
trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
10,6.427600
20,1.477800
30,1.441900
40,1.592700
50,1.598900
60,1.783700
70,1.389900
80,1.821200
90,1.800300
100,1.470200
